In [1]:
import pandas as pd
import requests

local imports

In [2]:
from main import make_query, parse_OTP_response
from utils import plot
from utils import get_config

set up

In [3]:
PATH = 'georequests_O(area10)_D.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

params = get_config('default_SUM.json')
# params = make_paths(params) # update file paths
# VoT - value of time, param

In [4]:
df = pd.read_csv(PATH)
df.head(3)

,Unnamed: 0,FID,x,y,adr_pelny,ogolem,zone_NO,inside_poly,probs,desti_zones,prob_dict,desti_zone,desti_x,desti_y,treq
0,1318,16943,19.871552,50.018840,JÓZEFA UNRUGA 25,6,188.0,True,"[0.015895665874688564, 0.006874302188204317, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[{1: 0.015895665874688564, 2: 0.00687430218820...",58,19.954225,50.045635,2024-03-28 07:54:33
1,3109,31851,19.873129,50.015181,BATALIONÓW CHŁOPSKICH 9G,3,188.0,True,"[0.015895665874688564, 0.006874302188204317, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[{1: 0.015895665874688564, 2: 0.00687430218820...",11,19.934195,50.070585,2024-03-28 07:52:19
2,1335,16997,19.885724,50.017353,MOCHNANIEC 2A,6,187.0,True,"[0.013729250247541647, 0.00609100536211645, 0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[{1: 0.013729250247541647, 2: 0.00609100536211...",88,19.896833,50.023390,2024-03-28 08:05:43


In [5]:
df1 = df.filter(['x','y','desti_x', 'desti_y', 'treq'], axis = 1).rename(
    columns = {'x': 'origin_x', 'y': 'origin_y', 'desti_x': 'destination_x', 'desti_y': 'destination_y'})

df1['treq'] = pd.to_datetime(df1.treq)
df1.head(3)

,origin_x,origin_y,destination_x,destination_y,treq
0,19.871552,50.018840,19.954225,50.045635,2024-03-28 07:54:33
1,19.873129,50.015181,19.934195,50.070585,2024-03-28 07:52:19
2,19.885724,50.017353,19.896833,50.023390,2024-03-28 08:05:43


<h3>query OTP for each request</h3>

In [6]:
df2 = df1.apply(lambda row: 
                 parse_OTP_response(requests.get(OTP_API, params=make_query(row.squeeze())).json()), axis=1)


In [7]:
# test
df2[0]

{'success': True,
 'n_itineraries': 3,
 'duration': 2458,
 'walkDistance': 1014.8020375854327,
 'transfers': 1,
 'transitTime': 1560,
 'waitingTime': 81,
 'modes': [['WALK', 478, 608],
  ['BUS', 240, 944],
  ['WALK', 101, 130],
  ['TRAM', 1320, 6352],
  ['WALK', 238, 274]]}

In [8]:
for name in list(df2[0].keys()):
    vals = []
    for i in range(len(df2)):
        if df2[i]['success'] == True:
            vals.append(df2[i][name])
        else:
            vals.append(None)
    df1[name] = vals
df1[:3]

,origin_x,origin_y,destination_x,destination_y,treq,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
0,19.871552,50.018840,19.954225,50.045635,2024-03-28 07:54:33,True,3.0,2458.0,1014.802038,1.0,1560.0,81.0,"[[WALK, 478, 608], [BUS, 240, 944], [WALK, 101..."
1,19.873129,50.015181,19.934195,50.070585,2024-03-28 07:52:19,True,3.0,2987.0,831.435541,1.0,2040.0,261.0,"[[WALK, 238, 293], [BUS, 240, 944], [WALK, 101..."
2,19.885724,50.017353,19.896833,50.023390,2024-03-28 08:05:43,True,3.0,826.0,839.163955,0.0,120.0,2.0,"[[WALK, 348, 398], [TRAM, 120, 1182], [WALK, 3..."


In [9]:
df1[61 : 64]

,origin_x,origin_y,destination_x,destination_y,treq,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
61,19.880008,50.015362,19.896833,50.023390,2024-03-28 07:48:47,True,3.0,1020.0,773.129173,1.0,300.0,87.0,"[[WALK, 242, 289], [BUS, 180, 582], [WALK, 35,..."
62,19.873445,50.015669,20.077178,50.072775,2024-03-28 08:02:50,None,NaN,NaN,NaN,NaN,NaN,NaN,None
63,19.870498,50.015416,19.896833,50.023390,2024-03-28 07:54:18,True,3.0,1172.0,1002.316571,1.0,360.0,21.0,"[[WALK, 334, 431], [BUS, 240, 944], [WALK, 101..."


In [10]:
# test
row = df1.sample(1).squeeze()
query = make_query(row)
r = requests.get(OTP_API, params=query)
route = parse_OTP_response(r.json())
route


{'success': True,
 'n_itineraries': 3,
 'duration': 1222,
 'walkDistance': 951.7633232976444,
 'transfers': 1,
 'transitTime': 420,
 'waitingTime': 21,
 'modes': [['WALK', 265, 342],
  ['BUS', 240, 944],
  ['WALK', 101, 130],
  ['TRAM', 180, 1596],
  ['WALK', 415, 478]]}

In [11]:
# visualization
plot(r.json()['plan'], color = 'green')

Trip from (19.8774,50.0138) to (19.9066,50.0301) at 1711609200000. 
3 connections found. 
Best one is 20min (952m walk, 1 transfer(s), wait time 0.35min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	WALK	342	265
2	BUS	944	240
3	WALK	131	101
4	TRAM	1596	180
5	WALK	478	415


In [12]:
def PT_utility(requests, params):
    if 'walkDistance' in requests.columns:
        requests = requests
        requests['PT_fare'] = 1 + requests.transitTime * params.avg_speed/1000 * params.ticket_price
        requests['u_PT'] = requests['PT_fare'] + \
                           params.VoT * (params.walk_factor * requests.walkDistance / params.speeds.walk +
                                           params.wait_factor * requests.waitingTime +
                                           params.transfer_penalty * requests.transfers + requests.transitTime)
    return requests

In [13]:
reqs = PT_utility(df1, params)

In [14]:

reqs

,origin_x,origin_y,destination_x,destination_y,treq,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes,PT_fare,u_PT
0,19.871552,50.018840,19.954225,50.045635,2024-03-28 07:54:33,True,3.0,2458.0,1014.802038,1.0,1560.0,81.0,"[[WALK, 478, 608], [BUS, 240, 944], [WALK, 101...",2.638,16.334679
1,19.873129,50.015181,19.934195,50.070585,2024-03-28 07:52:19,True,3.0,2987.0,831.435541,1.0,2040.0,261.0,"[[WALK, 238, 293], [BUS, 240, 944], [WALK, 101...",3.142,18.709041
2,19.885724,50.017353,19.896833,50.023390,2024-03-28 08:05:43,True,3.0,826.0,839.163955,0.0,120.0,2.0,"[[WALK, 348, 398], [TRAM, 120, 1182], [WALK, 3...",1.126,6.455123
3,19.874427,50.017614,19.895333,50.085894,2024-03-28 07:49:17,True,3.0,3772.0,1305.296789,2.0,2340.0,397.0,"[[WALK, 487, 622], [BUS, 240, 944], [WALK, 101...",3.457,25.540231
4,19.872725,50.014630,19.932209,50.056612,2024-03-28 08:03:51,True,3.0,2566.0,1369.558271,1.0,1260.0,174.0,"[[WALK, 362, 453], [BUS, 360, 2800], [WALK, 8,...",2.323,17.690090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,19.866355,50.022343,19.951528,50.063938,2024-03-28 07:59:35,True,3.0,2513.0,605.666977,1.0,1920.0,87.0,"[[WALK, 72, 81], [BUS, 360, 1973], [WALK, 35, ...",3.016,15.628057
162,19.873129,50.015181,19.861527,50.017915,2024-03-28 07:58:17,True,1.0,1039.0,1340.974000,0.0,0.0,0.0,"[[WALK, 1039, 1340]]",1.000,8.822348
163,19.870873,50.012687,19.896833,50.023390,2024-03-28 07:48:07,True,3.0,1314.0,1090.763737,1.0,420.0,21.0,"[[WALK, 416, 519], [BUS, 300, 1339], [WALK, 10...",1.441,11.170788
164,19.879112,50.019725,20.016993,50.086808,2024-03-28 08:01:18,True,3.0,3392.0,1440.863083,1.0,2160.0,32.0,"[[WALK, 751, 926], [BUS, 1980, 12806], [WALK, ...",3.268,21.207035


In [15]:
# reqs.to_csv("PT_utility_HD(area10).csv")